In [2]:
import pytesseract
import cv2

In [4]:
image = cv2.imread('DataForOCR/5.jpeg')
base_image = image.copy()

In [5]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [6]:
cv2.imwrite('test2/index_gray.png', gray)

True

In [114]:
blur = cv2.GaussianBlur(gray, (25,25), 3)

In [94]:
cv2.imwrite('test2/index_blur.png', blur)

True

In [123]:
thresh = cv2.threshold(blur, 0, 250, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [122]:
cv2.imwrite('test2/index_thresh.png', thresh)

True

In [106]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,7))

In [108]:
cv2.imwrite('test2/index_kernel.png', kernel)

True

In [109]:
dilate = cv2.dilate(thresh, kernel, iterations=1)

In [110]:
cv2.imwrite('test2/index_dilate.png', dilate)

True

In [74]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [75]:
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [76]:
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [78]:
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 20 and w > 5:
        roi = image[y:y+h, x:x+h]
        cv2.imwrite('test2/index_roi.png', roi)
        cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
cv2.imwrite('test2/index_bbox.png', image)

True

In [35]:
from matplotlib import pyplot as plt

#ret,thresh4 = cv2.threshold(gray,150,255,cv2.THRESH_TOZERO)
#ret,thresh4 = cv2.threshold(gray,160,255,cv2.THRESH_TOZERO)

ret,thresh4 = cv2.threshold(gray,150,255,cv2.THRESH_TOZERO)
cv2.imwrite('no_noise/thresh4.png', thresh4)



True

In [83]:
# Load image, grayscale, Otsu's threshold
image = cv2.imread('DataForOCR/0.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray,150,255,cv2.THRESH_TOZERO)
# Morph open to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

# Find contours and remove small noise
cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 50:
        cv2.drawContours(opening, [c], -1, 0, -1)

# Invert and apply slight Gaussian blur
result = 255 - opening
result = cv2.GaussianBlur(result, (3,3), 0)

# Perform OCR
#data = pytesseract.image_to_string(result, lang='eng', config='--psm 6')
#print(data)

cv2.imwrite('no_noise/thresh.png', thresh)
cv2.imwrite('no_noise/opening.png', opening)
cv2.imwrite('no_noise/result.png', result)

True

In [52]:
image = cv2.imread('test3/9.png')

data = pytesseract.image_to_string(image, lang='rus', config='--psm 6 -c tessedit_char_whitelist=АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя')
print(data)

о Ее
о о и
г россии Екея ЕДеРАЗЦИЯ
ОТДЕЛОМУФМСРОЗСНи
а Паспортзылан ы вв с
ПОСВЕРДЛОВСКОЙОБЛАСТИ
КИРОВСКОМРЕНЕГОРЕКАТЕРИНЬГЬЛ
о ВА ВЯНЗЕ
ЕН Б с
а Дитавалачи Кодаачавааделения т
я
о ИОН
а С к НЕ ь Со Е
ОЖА Е
и БЫ
ти а аа
ао я ЛичвыитрА
к сясВЕ ее ес
ПСХИ ры
ввь
г Е
а к
скроескофовковосоо АКАЕВА
аа Фамплия
й АИ Е Р
оС ыы р
ес Ще ЖАННА с
г в с зщ
в В РИН АНАТОЛЬЕТА
ея е Я
сЖЕЙвмиОЭ Г
у м
Е южиныРОР СВЕРДЛОВСК
я ры с
Но
ЕН Е
ВтБЫН
ЗЧ ть а в и Се ь



In [80]:
import numpy as np
image_mask = cv2.imread('DataForOCR/9.jpeg')
hsv = cv2.cvtColor(image_mask, cv2.COLOR_BGR2HSV)
low = np.array([117, 234, 228])
high = np.array([56, 112, 109])
mask = cv2.inRange(hsv, high, low)

cv2.imwrite('no_noise/image_mask.png', mask)


True

In [170]:
from natasha import NamesExtractor, doc, PER, MorphVocab
morph_vocab = MorphVocab()

extractor = NamesExtractor(morph_vocab)

text = ['ПСКОВСКОЙОБЛ', 'БОРОУСЫ', 'Месть', 'рождения', 'Хата', 'ЭчО', 'ДЕРБОРОУСЫ', 'отчество', 'ЮРЬЕВИЧ', 'Имя', 'РОМАН', 'МАКАРОВ']
for i in text:
    #print(i)
    matches = extractor(i.lower())

    for match in matches:
        print(match.fact)


Name(first='роман', last=None, middle=None)
Name(first='макаров', last=None, middle=None)


In [1]:
from yargy.tokenizer import MorphTokenizer
from yargy import rule, Parser, or_, and_, not_
from yargy.predicates import eq, gram
from yargy.interpretation import fact
from yargy.relations import gnc_relation


text = ['ДОНАЛЬД', 'джон', 'МУХб', 'ТГОРкУИНС', 'ШТАТНЬЮЙОРК', 'сил', 'ТРАМП', 'РЕНИ', 'фито', 'фиды', 'аАИ']
TOKENIZER = MorphTokenizer()
for i in text:
    print(list(TOKENIZER(i.lower())))


[MorphToken(value='дональд', span=[0, 7), type='RU', forms=[Form('дональд', Grams(NOUN,Name,anim,masc,nomn,sing))])]
[MorphToken(value='джон', span=[0, 4), type='RU', forms=[Form('джон', Grams(NOUN,Name,anim,masc,nomn,sing))])]
[MorphToken(value='мухб', span=[0, 4), type='RU', forms=[Form('мухб', Grams(NOUN,Orgn,inan,masc,nomn,sing)), Form('мухб', Grams(NOUN,Orgn,accs,inan,masc,sing)), Form('мухбый', Grams(ADJS,Qual,masc,sing)), Form('мухбнуть', Grams(VERB,indc,intr,masc,past,perf,sing)), Form('мухб', Grams(ADVB)), Form('мухб', Grams(ADJF,Abbr,Fixd,masc,nomn,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,gent,masc,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,datv,masc,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,accs,masc,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,ablt,masc,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,loct,masc,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,femn,nomn,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,femn,gent,sing)), Form('мухб', Grams(ADJF,Abbr,Fixd,datv,femn,sing)), Form('мухб'

In [168]:

COMPANY = rule(or_
                (and_
                (gram('Name'), 
                not_
                (gram('Fixd')), 
                not_
                (gram('Abbr'))), 
                and_
                (gram('Patr'), 
                not_
                (gram('Fixd')), 
                not_
                (gram('Abbr'))), 
                and_(gram('Surn'), 
                not_
                (gram('Fixd')),
                not_
                (gram('Abbr')))))
company_finder = rule(COMPANY)
parser_company = Parser(company_finder)
companies = []
for i in text:

    matches = (list(parser_company.findall(i)))
    for i in matches:
        companies.append([k.value for k in i.tokens])

print(companies)

[['ВИКТОРИЯ'], ['НИКОЛАЕВНА'], ['ШАПОШНИКОВА']]


In [186]:
import nltk
import pymorphy2

# probability score threshold
prob_thresh = 0.4

morph = pymorphy2.MorphAnalyzer()

text = """
['аЖАННА', 'машеВАНАТОЛЬЕВИА', 'тпЖЕНвны', 'СВЕРДЛОВСК', 'ЖАННА', 'Анчныйкол', 'Датавылачи', 'КИРОВСКОМ', 'ВКАТЕРИНЬ', 'Паспортвылан', 'ОБЛАС', 'ОТАВЯХТ', 'ьСВЕРДАЮВСКОЙ']"""

for word in nltk.word_tokenize(text):
    for p in morph.parse(word):
        print(p)
    

Parse(word='[', tag=OpencorporaTag('PNCT'), normal_form='[', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), '['),))
Parse(word="'ажанна", tag=OpencorporaTag('NOUN,anim,femn,Name sing,nomn'), normal_form="'ажанна", score=0.7142857142857143, methods_stack=((DictionaryAnalyzer(), 'жанна', 69, 0), (UnknownPrefixAnalyzer(score_multiplier=0.5), "'а")))
Parse(word="'ажанна", tag=OpencorporaTag('ADJS,Qual femn,sing'), normal_form="'ажанный", score=0.28571428571428575, methods_stack=((FakeDictionary(), "'ажанна", 345, 28), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'жанна')))
Parse(word="'", tag=OpencorporaTag('PNCT'), normal_form="'", score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), "'"),))
Parse(word=',', tag=OpencorporaTag('PNCT'), normal_form=',', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), ','),))
Parse(word="'машеванатольевиа", tag=OpencorporaTag('NOUN,inan,neut,Sgtm,Fixd,Orgn sing,nomn'), normal_form="'машеванатольевиа", score=0.16

In [ ]:
# Filter out single-character & non-alphabet predictions
# Print some meaningful results

pprint([inner_item
        for item in group_text
        for inner_item in item
        if len(inner_item) >= 2 and inner_item.isalpha()])